In [1]:
import os
from dotenv import load_dotenv
import openai

_ = load_dotenv(override=True)

In [2]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    api_key = os.getenv("OPEN_AI_KEY"),
    model = os.getenv("LLM_MODEL"),
    temperature=0.0
)

Tools for qagent

In [6]:
from langchain.agents import load_tools, initialize_agent, AgentType

tools = load_tools(["llm-math", "wikipedia"], llm = llm)

initialize Agent and ask a mathematical question

In [9]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

agent("What is derivative of x^2/2 + x evaluated in 2?")



> Entering new AgentExecutor chain...
To find the derivative of the function \(f(x) = \frac{x^2}{2} + x\), we can use the power rule and the sum rule of differentiation. The power rule states that the derivative of \(x^n\) is \(nx^{n-1}\), and the sum rule states that the derivative of the sum of two functions is the sum of their derivatives.

So, let's find the derivative of each term separately:

\(\frac{d}{dx} \left(\frac{x^2}{2}\right) = \frac{1}{2} \cdot 2x = x\)

\(\frac{d}{dx} (x) = 1\)

Now, we can add the derivatives together:

\(\frac{d}{dx} \left(\frac{x^2}{2} + x\right) = x + 1\)

To evaluate this derivative at \(x = 2\), we substitute \(x = 2\) into the derivative:

\(x + 1 = 2 + 1 = 3\)

Final Answer: The derivative of \(f(x) = \frac{x^2}{2} + x\) evaluated at \(x = 2\) is 3.

> Finished chain.


{'input': 'What is derivative of x^2/2 + x evaluated in 2?',
 'output': 'The derivative of \\(f(x) = \\frac{x^2}{2} + x\\) evaluated at \\(x = 2\\) is 3.'}

No a wikipedia question

In [12]:
question = "Tom Morello is an guitarrist which bands had he been part of?"
result = agent(question) 



> Entering new AgentExecutor chain...
Question: Which bands has Tom Morello been a part of?
Thought: I can use Wikipedia to find information about Tom Morello's bands.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Tom Morello"
}
```
Observation: Page: Tom Morello
Summary: Thomas Baptist Morello (born May 30, 1964) is an American guitarist, singer, songwriter, and political activist. He is best known for his tenure with the rock bands Rage Against the Machine and Audioslave. Between 2016 and 2019, Morello was a member of the supergroup Prophets of Rage. Morello was a touring musician with Bruce Springsteen and the E Street Band. Under the moniker the Nightwatchman Morello released his solo work. Together with Boots Riley, he formed Street Sweeper Social Club. Morello co-founded Axis of Justice, which airs a monthly program on Pacifica Radio station KPFK (90.7 FM) in Los Angeles.
Born in Harlem, New York, and raised in Libertyville, Illinois, Morello became interested in mu

# Custom Tool

In [14]:
from langchain.agents import tool
from datetime import date

In [15]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [16]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [17]:
result = agent("whats the date today?")



> Entering new AgentExecutor chain...
Question: What's the date today?
Thought: I can use the `time` tool to get the current date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2023-11-10
Thought:Could not parse LLM output: I now know the final answer.
Observation: Invalid or incomplete response
Thought:I made a mistake in my previous response. I apologize for the confusion. Let me correct it and provide the final answer.

Question: What's the date today?
Thought: I can use the `time` tool to get the current date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```


Observation: 2023-11-10
Thought:I now know the final answer.
Final Answer: The date today is 2023-11-10.

> Finished chain.
